# Определение закономерностей, влияющих на успешность компьютерных игр  интернет-магазина «Стримчик»

## Описание проекта

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Не важно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.

## Описание данных
- Name — название игры;
- Platform — платформа;
- Year_of_Release — год выпуска;
- Genre — жанр игры;
- NA_sales — продажи в Северной Америке (миллионы долларов);
- EU_sales — продажи в Европе (миллионы долларов);
- JP_sales — продажи в Японии (миллионы долларов);
- Other_sales — продажи в других странах (миллионы долларов);
- Critic_Score — оценка критиков (максимум 100);
- User_Score — оценка пользователей (максимум 10);
- Rating — возрастной рейтинг ESRB.

## План работы

1. Загрузка и предобработка <br>
1.1. Ознакомление с данными <br>
1.2. Замена названий столбцов<br>
1.3. Обработка пропусков и дубликатов<br>
1.4. Суммарные продажи во всех регионах<br>
2. Исследовательский анализ данных<br>
2.1. Количество игр, выпускавшиеся в разные годы<br>
2.2. Анализ продаж по платформам с наибольшими суммарными продажами<br>
2.3. Срез данных за актуальный период<br>
2.4. Платформы, лидирующие по продажам<br>
2.5. Глобальные продажи игр в разбивке по платформам<br>
2.6. Влияние на продажи внутри PS3 отзывов пользователей и критиков<br>
2.7. Общее распределение игр по жанрам<br>
3. Портрет пользователя каждого региона
2.1. Самые популярные платформы (топ-5)<br>
2.2. Самые популярные жанры (топ-5<br>
2.3. Влияние рейтинга ESRB на продажи в отдельном регионе<br>
4. Проверка гипотез<br>
4.1. Средние пользовательские рейтинги платформ Xbox One и PC одинаковые<br>
4.2. Средние пользовательские рейтинги жанров Action и Sports разные<br>
5. Общий вывод<br>

In [61]:
import datetime
import numpy as np
import pandas as pd

from scipy import stats as st

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

## 1. Загрузка и предобработка

### 1.1. Ознакомление с данными

In [62]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
Name               16713 non-null object
Platform           16715 non-null object
Year_of_Release    16446 non-null float64
Genre              16713 non-null object
NA_sales           16715 non-null float64
EU_sales           16715 non-null float64
JP_sales           16715 non-null float64
Other_sales        16715 non-null float64
Critic_Score       8137 non-null float64
User_Score         10014 non-null object
Rating             9949 non-null object
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [63]:
df['Genre'].value_counts()

Action          3369
Sports          2348
Misc            1750
Role-Playing    1498
Shooter         1323
Adventure       1303
Racing          1249
Platform         888
Simulation       873
Fighting         849
Strategy         683
Puzzle           580
Name: Genre, dtype: int64

In [65]:
df['Year_of_Release'].value_counts().head(5)

2008.0    1427
2009.0    1426
2010.0    1255
2007.0    1197
2011.0    1136
Name: Year_of_Release, dtype: int64

In [66]:
df['Platform'].value_counts().head(5)

PS2     2161
DS      2151
PS3     1331
Wii     1320
X360    1262
Name: Platform, dtype: int64

In [68]:
df['Rating'].value_counts()

E       3990
T       2961
M       1563
E10+    1420
EC         8
RP         3
K-A        3
AO         1
Name: Rating, dtype: int64

Всего в таблице 16 715 объектов и 11 признаков для них. На примере первых 5 записей убедились, что данные распарсились правильно. Пока видно что пропуски имеются в признаках с рейтингом, в остальном данные в аккуртном виде, большинство типа object и float64.

Самый популярный жанр Action, больше всего игр выпущено в 2008-2009 годах, самые популярные платформы PS2 и DS, имеется примерно 4 тысячи игр с рейтингом E.

### 1.2. Замена названий столбцов

In [ ]:
df.columns = df.columns.str.lower()
df.columns

Все названия стобцов привели к нижнему регистру.

### 1.3. Обработка пропусков и дубликатов

In [ ]:
df.isna().sum(axis=0).sort_values(ascending=False)/len(df)

Убедились и поняли, что почти половины данных пропущены в признаках critic_score, rating, user_score.  
Имеются также пропуски в year_of_release, genre, name.   

In [ ]:
df.duplicated().sum()

Дубликатов не было найдено.

In [ ]:
df.year_of_release = df.year_of_release.astype('Int64')
df.year_of_release.head()

Поменяли тип признака year of release с вещественного на целочисленный, потому что год не может быть дробным.

In [ ]:
dict_year_of_release = df.set_index('name')['year_of_release'].to_dict()
dict_critic_score = df.groupby('name')['critic_score'].median().to_dict()
dict_user_score = df.set_index('name')['user_score'].to_dict()
dict_rating = df.set_index('name')['rating'].to_dict()

df['year_of_release'] = df['year_of_release'].fillna(df['name'].map(dict_year_of_release))
df['critic_score'] = df['critic_score'].fillna(df['name'].map(dict_critic_score))
df['user_score'] = df['user_score'].fillna(df['name'].map(dict_user_score))
df['rating'] = df['rating'].fillna(df['name'].map(dict_rating))

df.isna().sum(axis=0).sort_values(ascending=False)/len(df)

Заполнили часть пропусков по названию игры, предварительно создав словарь на основе имеющихся данных.

In [ ]:
df.user_score.describe() #mean68.9 median71

df.loc[df['user_score']=="tbd", 'user_score'] = None
df['user_score'] = df['user_score'].astype('float')
df.info()

Aббреватиара tbd расшифровыается как "to be determined" и означает, что количество отзывов недостаточно для подсчета общего рейтинга, то есть по сути получается, что у игры нет рейтинга, поэтому их заменили на None и сконвертировали в вещественный тип для последующих вычислительных операций.

### 1.4. Суммарные продажи во всех регионах

In [ ]:
df['total_sales'] = df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df.head()

Посчитали итоговые продажи по всем регионам и сохранили в новом столбце total_sales. Данные готовы к исследованию.

## 2. Исследовательский анализ данных

### 2.1. Количество игр, выпускавшиеся  в разные годы

In [ ]:
df.year_of_release.describe()
df_by_year = df.pivot_table(index = 'year_of_release', values = 'name', aggfunc = 'count').reset_index()
df_by_year.tail()

In [ ]:
df_by_year = df_by_year.query('year_of_release > 1995')
sns.catplot(x = 'year_of_release', y='name', data = df_by_year, kind = 'bar', aspect = 2)
plt.show()

До 1995 выпускалось всего примерно по 100 игр (поэтому не берем в расчет), затем произошла технологическая революция и с 2005 года выпускалось более 800 игр, пик был достигнут примерно в 2008-2009 годах (1 400 игр выпускалось ежегодно) и затем призошел спад: до 600 игр в 2015 году.

### 2.2. Анализ продаж по платформам с наибольшими суммарными продажами

In [ ]:
platforms = df.pivot_table(index=['platform'], values='total_sales', aggfunc='sum').reset_index()
platforms.total_sales.describe()
platforms = platforms.query('total_sales > 900').reset_index()
platforms = platforms['platform']
platforms

In [ ]:
df_by_platform = (df
                  .query('platform in @platforms')
                  .pivot_table(index=['platform', 'year_of_release'], values='total_sales', aggfunc='sum')
                  .reset_index())
df_by_platform.head()

In [ ]:
sns.set_style("whitegrid")
a = sns.catplot(x = 'year_of_release', y = 'total_sales', hue ='platform', kind="bar", 
                alpha=0.75, data=df_by_platform, aspect=2.5)

- Платформы с наибольшими суммарными продажами это PS2, PS3, Wii и X360
- Средняя жизненный цикл длится 8-10 лет
- Пик популярности платформы приходится на середину жизненного цикла

### 2.3. Срез данных за актуальный период

In [ ]:
df_after_2011 = df.query('year_of_release >= 2011').reset_index()
df_after_2011.head()

Нам интересно исследовать данные только активно развивающиеся платформы и так как пик приходится на половину жизненного цикла, то взли срез с 2011 по 2016 год.

### 2.4. Платформы, лидирующие по продажам

In [ ]:
leading_platforms = (df_after_2011.pivot_table(index=['platform'], values='total_sales', aggfunc=['count','sum'])
                  .reset_index())
leading_platforms.columns = ['platform', 'sales_count', 'sales_sum']
leading_platforms = (leading_platforms.query('sales_count > 150 and sales_sum > 150')
                     .sort_values(by = 'sales_sum', ascending=False).reset_index())
leading_platforms.style.bar(subset=['sales_count', 'sales_sum'], color='bisque')

In [ ]:
leading_platforms = leading_platforms['platform']
df_by_leading_platforms = df_after_2011.query('platform in @leading_platforms')
df_by_leading_platforms.head()
leading_platforms

In [ ]:
a = sns.catplot(x = 'year_of_release', y = 'total_sales', hue ='platform', kind="bar", 
                alpha=0.75, data=df_by_leading_platforms, aspect=2.5)

- Лидеры по продажам — PS3, X360, PS4, 3DS, Xone
- PS3, X360 хоть и являются самыми прибыльными, но в последние года практически не продаются и значит считать их актуальными нельзя
- Также с 2013 новый игроки и лидеры на рынке — PS4 и XOne

### 2.5. Глобальные продажи игр в разбивке по платформам

In [ ]:
df_by_leading_platforms_and_sales = df_by_leading_platforms.query('total_sales < 2.5')
df_by_leading_platforms_and_sales.head()

In [ ]:
plt.figure(figsize=(10, 6))
a = sns.boxplot(x="platform", y="total_sales", data=df_by_leading_platforms_and_sales, palette='coolwarm')

Судя по всему равномерная прибыль не со всех игр, а с самых популярных, потому что большинство игр продаются не очень активно. 
Скорее всего единичные игры становятся популярным и за счет них компании зарабатывают, поэтому так много и выбросов.
Лучше всего продаются игры на платформах PS3 и X360, остальные примерно на одинаковом уровне (от 2 до 6)

### 2.6. Влияние на продажи внутри PS3 и X360 отзывов пользователей и критиков

In [ ]:
ps3 = df_after_2011.query('platform == "PS3"')
cols = ['critic_score', 'user_score', 'total_sales']
a = sns.pairplot(ps3[cols])

In [ ]:
print('Корреляция с мнением критиков', ps3['total_sales'].corr(ps3['critic_score']).round(2))
print('Корреляция с мнением пользователей',ps3['total_sales'].corr(ps3['user_score']).round(2))

In [ ]:
x360 = df_after_2011.query('platform == "X360"')
cols = ['critic_score', 'user_score', 'total_sales']
a = sns.pairplot(x360[cols])

In [ ]:
print('Корреляция с мнением критиков', x360['total_sales'].corr(x360['critic_score']).round(2))
print('Корреляция с мнением пользователей', x360['total_sales'].corr(x360['user_score']).round(2))

- Продажи зависят от рейтингов: в первую очередь от оценок критиков (в среднем 0,35)
- Оценки пользователей тоже влияют, но в данном случае корреляция ниже более чем в 2 раза (в среднем 0,03)

### 2.7. Общее распределение игр по жанрам

In [ ]:
df_by_genre = (df_after_2011
               .pivot_table(index='genre', values='total_sales', aggfunc='sum')
               .sort_values(by='total_sales', ascending=False).reset_index())
a = sns.catplot(x='genre', y='total_sales', data=df_by_genre, kind='bar', aspect=3)

- Самый прибыльный жанр Action выделяется среди своих конкурентов — продажи (6 000) на порядок выше остальных
- Далее почти на одинаковом уровне от 2 200 до 3 200 находятся жанры Sports, Misc, Role-Playing 
- Менее популярные жанры Strategy и Puzzle (700-800 игр)

In [ ]:
df__expensive_games = df_after_2011.query('total_sales > 1')
df_by_genre_and_expensive_games = (df__expensive_games
               .pivot_table(index='genre', values='total_sales', aggfunc='sum')
               .sort_values(by='total_sales', ascending=False).reset_index())
a = sns.catplot(x='genre', y='total_sales', data=df_by_genre_and_expensive_games, kind='bar', aspect=3)

Было интересно еще рассмотреть рейтинг жанров по самым дорогим играм - это также можно назвать популярностью. По результатам, жанр Shooter вырвался вперед.

## 3. Портрет пользователя каждого региона 

### 3.1. Самые популярные платформы (топ-5)

In [ ]:
df_regional_platforms = (df_after_2011
                         .pivot_table(index='platform', values=['na_sales','eu_sales', 'jp_sales'], aggfunc='sum'))
df_regional_platforms.plot(kind='bar', grid=True, title='Продажи в регионах по платформам', figsize = (15, 5))
plt.show()

- Больше всего покупают в Северной Америке, затем в Европе и в Японии
- Европейцы и американцы имеют схожие предпочтения, им интересны платформы PS3, PS4 и X360
- Самая популярная платформа в Японии — 3DS

### 3.2. Самые популярные жанры (топ-5)

In [ ]:
for column in ['na_sales','eu_sales', 'jp_sales']:
    (df_after_2011
     .pivot_table(index='genre', values=column, aggfunc='sum')
     .sort_values(by=column, ascending=False).reset_index().head(5)
     .plot(x = 'genre', y = column, kind='bar'))

Картина по жанрам имеет те же характеристики: 
- Вновь предпочтения американцев и европейцев совпадают — жанры Action и Sports.
- У японцев жанр Action следует только после лидера Role-Playing, что вполне вяжется с их культурным особенностями.

### 3.3. Влияние рейтинга ESRB на продажи в отдельном регионе

In [ ]:
for column in ['na_sales','eu_sales', 'jp_sales']:
    (df_after_2011
     .pivot_table(index='rating', values=column, aggfunc='sum')
     .sort_values(by=column, ascending=False).reset_index().head(5)
     .plot(x = 'rating', y = column, kind='bar'))

- В Америке и Европе большинство выбирают игры рейтинга M (от 17 лет) и E (для всех)
- В Японии самый популярный рейтинг E и категория Teen (от 13 лет)

Итого:

- Больше всего покупают в Северной Америке, затем в Европе и в Японии

- Европейцы и американцы имеют схожие предпочтения, им интересны платформы PS3, PS4, X360, игры жанров Action и Sports. Большинство выбирают игры рейтинга M (от 17 лет) и E (для всех)

- Самая популярная платформа в Японии — 3DS. У японцев жанр Action следует только после лидера Role-Playing, что вполне вяжется с их культурным особенностями.

## 4. Проверка гипотез

### 4.1. Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;

In [ ]:
xbox = df_after_2011.query('platform == "XOne"').reset_index()
xbox = xbox[['user_score']].dropna()
pc = df_after_2011.query('platform == "PC"').reset_index()
pc = pc[['user_score']].dropna()

#H0 — средние пользовательские рейтинги платформ Xbox One и PC одинаковые
#H1 — редние пользовательские рейтинги различаются

alpha = .05
results = st.ttest_ind(xbox, pc, equal_var = False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

Тест показал, что вероятнее всего нулевая гипотеза правильна и пользовательские рейтинги платформ схожи

### 4.2. Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

In [ ]:
action = df_after_2011.query('genre== "Action"').reset_index()
action = action[['user_score']].dropna()
sports = df_after_2011.query('genre == "Sports"').reset_index()
sports = sports[['user_score']].dropna()

#H0 — средние пользовательские рейтинги жанров Action и Sports одинаковые
#H1 — редние пользовательские рейтинги различаются

alpha = .05
results = st.ttest_ind(action, sports, equal_var = False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

В данном случае нулевую гипотезу отвергаем, соответственно, скорее всего, пользовательские рейтинги жанров Action и Sports различаются.

## 5. Общий вывод

До 1995 выпускалось примерно по 100 игр, затем произошла технологическая революция и с 2005 года выпускалось более 800 игр, пик был достигнут примерно в 2008-2009 годах (1 400 игр выпускалось ежегодно) и затем призошел спад: до 600 игр в 2015 году. Платформы с наибольшими суммарными продажами это DS, группы платформ PS, WII и X360. Средняя жизненный цикл длится 8-10 лет пик популярности платформы приходился на 4-5 год.

Решили исследовать данные за период, на который приходится пик популярности платформе, то есть за половину жизненного цикла с 2011 по 2016 года. Лидеры по продажам — PS3, X360, PS4, 3DS, Xone. В то же время хоть PS3, X360 и являются самыми прибыльными, но в последние года практически не продаются. Также с 2013 новый лидеры на рынке — PS4 и XOne

Самый прибыльный жанр Action выделяется среди своих конкурентов — продажи (6 000) на порядок выше остальных Далее почти на одинаковом уровне от 2 200 до 3 200 находятся жанры Sports, Misc, Role-Playing и иенее популярный жанры Strategy и Puzzle (700-800).

Продажи зависят от рейтингов: в первую очередь от оценок критиков (в среднем 0,35). Оценки пользователей тоже влияют, но в данном случае корреляция низкая (в среднем 0,03). Возможно, это объясняется тем, что наша статистическая гипотеза подтвердилась: пользовательские рейтинги платформ Xbox и PC схожи, при этом рейтинги жанров Action и Sports отличаются.

Портреты пользователей каждого регоина:
- Больше всего покупают в Северной Америке, затем в Европе и в Японии 
- Европейцы и американцы имеют схожие предпочтения, им интересны платформы PS3, PS4, X360, игры жанров Action и Sports. Большинство выбирают игры рейтинга M (от 17 лет) и E (для всех)
- Самая популярная платформа в Японии — 3DS. У японцев жанр Action следует только после лидера Role-Playing, что вполне вяжется с их культурным особенностями.

Рекомендуется выпускать игры для платформ PS4, XOne как платформ находящихся на пике жизненного цикла и в жанре Action как самом популярном. А также жанр Sports для американского и европейского рынка и жанры Role-Playing для Японии.
Стоит также следить за рейтингами от критиков, так как от него напрямую зависят продажи.